Import basic Libraries

In [ ]:
import pandas as pd
import math
import numpy as np

Import the training file

In [ ]:
train_file = '../input/pubg-finish-placement-prediction/train_V2.csv'

Read the training file into a dataframe

In [ ]:
df = pd.read_csv(train_file)
df.head()

Checking the datatypes of all columns in the dataframe

In [ ]:
df.dtypes

Dropping the 3 columns that have less significance over the others because the unique numbers don't play a heavy role in the modelling process.

In [ ]:
df.drop(['Id','groupId','matchId'], axis = 1, inplace = True)
df.head()

Also in the above datatypes, the feature 'matchType' is of object datatype.

This type of data cannot be passed in the model.

In [ ]:
mT = df.matchType.unique()
mT

So we assign all the unique values in the a unique code to categorize them. 

In [ ]:
mType = {}
c = 0
for i in range(len(mT)):
    mType.update({mT[i] : int(c)})
    c += 1
    
mType

After we assign these values, we map these values with the above dictionary.

In [ ]:
 df['matchType'] = df['matchType'].map(mType)

I'm splitting the data into 20 % of the total data because the training takes too long on my local machine. If you choose to wait for longer, please select a bigger dataset. 

In [ ]:
# perc = int(input("What percent of the data do you want? : "))
# The input was given as 20 
perc = 20
datasize = df.shape[0]
size = math.ceil((perc/100)*datasize)
train_df = df[:size]
#Taking 20% of data

In [ ]:
train_df

Checking the Target variable - winPlacePerc. 

In [ ]:
print(train_df.winPlacePerc.min(), train_df.winPlacePerc.max())

Taking 310606 elements in the training data as a validation dataset.
Why such a specific number? 

Only because I wanted to end my training dataset at 1200000th datapoint in the dataset. (Some Random OCD) 

In [ ]:
valid_df = df[889394:1200000]
valid_df.head()

In [ ]:
valid_df.shape

Importing our Algorithm, the RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

This is just assiging a variable to calling the function. 


The (n_jobs = -1) means - "Hey CPU, I'm the RandomForest. Give me all your cores for this dataset training. Or I'll dismiss you (Not really!)."

In [ ]:
m = RandomForestRegressor(n_jobs = -1)

Split the dataset into Dependent Features and Independent Features.

X_train is Independent.

y_train is dependent.

In [ ]:
X_train = train_df.drop(train_df.columns[-1] , axis = 1)
y_train = train_df[train_df.columns[-1]]

FIT IT!

In [ ]:
%time m.fit(X_train, y_train) 
#with 20% of total data takes 8 mins

This is an inbuilt function in RF. This check the R-Squared score of the fitting of the model. 

Don't worry much about it this is your first code.

Worry a lot about it if this is your 100'th code. 

In [ ]:
m.score(X_train, y_train)

We create a Independent variable dataset by dropping the dependent variable from the dataset (Only for the time being).

In [ ]:
X_valid = valid_df.drop(valid_df.columns[-1], axis = 1)
X_valid

Predict the validation values. 

In [ ]:
valid_preds = m.predict(X_valid)

Append all the ACTUAL values in the Validation dataset to an array.

In [ ]:
actual_validation_array = []

for i in valid_df.winPlacePerc:
    actual_validation_array.append(i)

Just printing the values.

In [ ]:
print(actual_validation_array[0], valid_preds[0])

In [ ]:
if len(actual_validation_array) == len(valid_preds) : print("Array size is same")

Calculating RMSE of the predicted values and the actual values. 

In [ ]:
squared_error = []
for i in range(len(valid_preds)):
    error = (actual_validation_array[i] - valid_preds[i])**2
    squared_error.append(error)
    
rmse = (sum(squared_error)/len(valid_preds))**(1/2)
print(f"RMSE of predicted Regression values to actual values in valid_df is : {round(rmse, 5)}")

Importing the Test Dataset.

In [ ]:
test_file = '../input/pubg-finish-placement-prediction/test_V2.csv'
df_test = pd.read_csv(test_file)
df_test.head()

Clipping the test dataset. Because, again, my computer isn't that powerful.

In [ ]:
df_test = df_test[:200000]
df_test.drop(['Id','groupId','matchId'], axis = 1, inplace = True)
df_test.head()

Checking types.

In [ ]:
df_test.dtypes

Mapping the matchType with the dictionary mType.

In [ ]:
df_test.matchType = df_test.matchType.map(mType)
df_test.head()

Calculate prediction on the Test Dataset.

In [ ]:
%time preds = m.predict(df_test)

Print the Predictions.

In [ ]:
preds

You are a God now. You just predicted the future of those PubG players without even looking at their dataset.

Go sip a beer now!